In [9]:
import requests

url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz"
response = requests.get(url, allow_redirects=True)

# Save to file
with open("green_tripdata_2019-10.csv.gz", "wb") as file:
    file.write(response.content)

print("Download complete!")

Download complete!


In [12]:
import json
import csv
from time import time
from kafka import KafkaProducer

# Function to serialize the message as JSON
def json_serializer(data):
    return json.dumps(data).encode('utf-8')

# Kafka configuration
server = "localhost:9092"
topic_name = "green-trips"

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

# Path to the CSV file
csv_file = "D:/OneDrive/Desktop/zoomcamp/zoomcamp_2025/week_6/2023-python-gsg/green_tripdata_2019-10/green_tripdata_2019-10.csv"

# Columns to keep
required_columns = [
    'lpep_pickup_datetime',
    'lpep_dropoff_datetime',
    'PULocationID',
    'DOLocationID',
    'passenger_count',
    'trip_distance',
    'tip_amount'
]

# Start timer
t0 = time()

# Read and send data to Kafka
with open(csv_file, 'r', newline='', encoding='utf-8') as file:
    reader = csv.DictReader(file)

    for row in reader:
        # Filter only the required columns
        message = {key: row[key] for key in required_columns}
        producer.send(topic_name, value=message)

# Ensure all messages are delivered
producer.flush()

# End timer
t1 = time()
took = t1 - t0

print(f"Time taken to send the dataset: {took:.2f} seconds")


Time taken to send the dataset: 128.09 seconds
